In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from pandas import Series, DataFrame

In [3]:
ad = pd.read_csv("Advertising(1).csv", header=0)

In [4]:
ad

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [5]:
ad['sales'] = (ad['sales'] > 10)*1

In [6]:
X = ad[['TV','radio','newspaper']]
Y = ad['sales']

In [7]:
from sklearn.linear_model import LogisticRegression 

In [201]:
from sklearn.model_selection import train_test_split
X_1,X_2,Y_1,Y_2=train_test_split(X,Y,test_size=0.5,random_state=0)

In [202]:
model = LogisticRegression()   # model instance setup
model1 = model.fit(X_1,Y_1)

In [203]:
Y_pred2 =model1.predict(X_2)     

In [204]:
metrics.confusion_matrix(Y_2, Y_pred2)

array([[19,  6],
       [ 1, 74]], dtype=int64)

In [205]:
1 - model1.score(X_2,Y_2)  

0.06999999999999995

In [206]:
model2 = model.fit(X_2,Y_2)      # model fitting for 2nd fold
Y_pred1 =model2.predict(X_1)     # prediction
metrics.confusion_matrix(Y_1, Y_pred1)

array([[19,  1],
       [ 2, 78]], dtype=int64)

In [207]:
1 - model2.score(X_1,Y_1)

0.030000000000000027

In [208]:
(pd.concat([Y_1,Y_2]) != np.concatenate([Y_pred1,Y_pred2])).mean()  # CV misclassification rate

0.05

## (1) 2-fold CV에서 LDA, QDA, KNN에 의한 오분류율을 각각 계산하여라.

In [209]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model_lda = LinearDiscriminantAnalysis()
model_lda2 = model_lda.fit(X_1,Y_1)
Y_pred_lda =model_lda2.predict(X_2)
metrics.confusion_matrix(Y_2, Y_pred_lda)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [210]:
cm1 = metrics.confusion_matrix(Y_2, Y_pred_lda) # confusion matrix
cm1

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [211]:
(cm1[1,0]+cm1[0,1]) / cm1.sum()  # lda 오분류율 

0.09

----

In [212]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model_qda = QuadraticDiscriminantAnalysis()
model_qda2 = model_qda.fit(X_1,Y_1)
Y_pred_qda =model_qda2.predict(X_2)
metrics.confusion_matrix(Y_2, Y_pred_qda)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [214]:
cm2 = metrics.confusion_matrix(Y_2, Y_pred_qda) # confusion matrix
cm2

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [215]:
(cm2[1,0]+cm2[0,1]) / cm2.sum()  # qda 오분류율 

0.09

----

In [216]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_1, Y_1)
Y_pred_knn =model_knn.predict(X_2)
metrics.confusion_matrix(Y_2, Y_pred_knn)

array([[20,  5],
       [ 2, 73]], dtype=int64)

In [217]:
cm3 = metrics.confusion_matrix(Y_2, Y_pred_knn) # confusion matrix
cm3

array([[20,  5],
       [ 2, 73]], dtype=int64)

In [218]:
(cm3[1,0]+cm3[0,1]) / cm3.sum()  # knn 오분류율 

0.07

---

## 5-fold CV를 이용하여 Logistic, LDA, QDA, KNN에 의한 오분류율을 각각 계산하여라.

In [38]:
ad_s = ad.sample(frac=1,random_state=2).reset_index(drop=True)
ad_s.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,113,175.7,15.4,2.4,1
1,30,70.6,16.0,40.8,1
2,183,56.2,5.7,29.7,0
3,200,232.1,8.6,8.7,1
4,194,166.8,42.0,3.6,1


In [39]:
N = len(ad_s)
N

200

In [42]:
n = N/5

In [43]:
n = int(n)

-----

In [485]:
ad1 = ad_s.iloc[:n,]
ad2 = ad_s.iloc[(n):2*n,]
ad3 = ad_s.iloc[(2*n):3*n,]
ad4 = ad_s.iloc[(3*n):4*n,]
ad5 = ad_s.iloc[(4*n):5*n,]

In [486]:
X = ad[['TV','radio','newspaper']]
Y = ad['sales']

In [487]:
from sklearn.model_selection import train_test_split
X_train_1,X_test_1,Y_train_1,Y_test_1=train_test_split(X,Y,test_size=0.5,random_state=0)

In [488]:
kfold_ad1_test = ad1
kfold_ad1_train=pd.concat([ad2,ad3,ad4,ad5])

In [489]:
X_train_1 = kfold_ad1_train[['TV','radio','newspaper']]
Y_train_1 = kfold_ad1_train['sales']
X_test_1 = kfold_ad1_test[['TV','radio','newspaper']]
Y_test_1 = kfold_ad1_test['sales']

In [490]:
model = LogisticRegression()   
model1 = model.fit(X_train_1,Y_train_1) 

In [491]:
Y_pred2 =model1.predict(X_test_1)     # prediction

In [492]:
metrics.confusion_matrix(Y_test_1, Y_pred2)

array([[ 7,  2],
       [ 1, 30]], dtype=int64)

In [493]:
cm1 = metrics.confusion_matrix(Y_test_1, Y_pred2) # confusion matrix
(cm1[1,0]+cm1[0,1]) / cm1.sum()  # lda  lohist오분류율 

0.075

----

In [494]:
kfold_ad2_test = ad2
kfold_ad2_train=pd.concat([ad5,ad1,ad3,ad4])
X_train_2 = kfold_ad2_train[['TV','radio','newspaper']]
Y_train_2 = kfold_ad2_train['sales']
X_test_2 = kfold_ad2_test[['TV','radio','newspaper']]
Y_test_2 = kfold_ad2_test['sales']
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(X,Y,test_size=0.5,random_state=0)

In [495]:
model = LogisticRegression()   
model2 = model.fit(X_train_2,Y_train_2) 
Y_pred2 =model2.predict(X_test_2)     # prediction

In [496]:
metrics.confusion_matrix(Y_test_2, Y_pred2)

array([[19,  6],
       [ 1, 74]], dtype=int64)

In [497]:
cm2 = metrics.confusion_matrix(Y_test_2, Y_pred2) # confusion matrix
(cm2[1,0]+cm2[0,1]) / cm2.sum() 

0.07

----

In [498]:
kfold_ad3_test = ad3
kfold_ad3_train=pd.concat([ad5,ad1,ad2,ad4])
X_train_3 = kfold_ad3_train[['TV','radio','newspaper']]
Y_train_3 = kfold_ad3_train['sales']
X_test_3 = kfold_ad3_test[['TV','radio','newspaper']]
Y_test_3 = kfold_ad3_test['sales']
X_train_3,X_test_3,Y_train_3,Y_test_3=train_test_split(X,Y,test_size=0.5,random_state=0)

In [499]:
model = LogisticRegression()   
model3 = model.fit(X_train_3,Y_train_3) 
Y_pred3 =model3.predict(X_test_3)

In [500]:
metrics.confusion_matrix(Y_test_3, Y_pred3)

array([[19,  6],
       [ 1, 74]], dtype=int64)

In [501]:
cm3 = metrics.confusion_matrix(Y_test_3, Y_pred3) # confusion matrix
(cm3[1,0]+cm3[0,1]) / cm3.sum() 

0.07

-----

In [502]:
kfold_ad4_test = ad4
kfold_ad4_train=pd.concat([ad5,ad1,ad2,ad3])
X_train_4 = kfold_ad4_train[['TV','radio','newspaper']]
Y_train_4 = kfold_ad4_train['sales']
X_test_4 = kfold_ad4_test[['TV','radio','newspaper']]
Y_test_4 = kfold_ad4_test['sales']
X_train_4,X_test_4,Y_train_4,Y_test_4=train_test_split(X,Y,test_size=0.5,random_state=0)

In [503]:
model = LogisticRegression()   
model4 = model.fit(X_train_4,Y_train_4) 
Y_pred4 =model4.predict(X_test_4)

In [506]:
metrics.confusion_matrix(Y_test_4, Y_pred4)

array([[19,  6],
       [ 1, 74]], dtype=int64)

In [507]:
cm4 = metrics.confusion_matrix(Y_test_4, Y_pred4) # confusion matrix
(cm4[1,0]+cm4[0,1]) / cm4.sum() 

0.07

-----

In [508]:
kfold_ad5_test = ad5
kfold_ad5_train=pd.concat([ad4,ad1,ad2,ad3])
X_train_5 = kfold_ad5_train[['TV','radio','newspaper']]
Y_train_5 = kfold_ad5_train['sales']
X_test_5 = kfold_ad5_test[['TV','radio','newspaper']]
Y_test5 = kfold_ad5_test['sales']
X_train_5,X_test_5,Y_train_5,Y_test_5=train_test_split(X,Y,test_size=0.5,random_state=0)

In [509]:
model = LogisticRegression()   
model5 = model.fit(X_train_5,Y_train_5) 
Y_pred5 =model5.predict(X_test_5)

In [510]:
metrics.confusion_matrix(Y_test_5, Y_pred5)

array([[19,  6],
       [ 1, 74]], dtype=int64)

In [511]:
cm5 = metrics.confusion_matrix(Y_test_5, Y_pred5) # confusion matrix
(cm5[1,0]+cm5[0,1]) / cm5.sum() 

0.07

### logist 5-fold 의 평균은 0.071 이다

In [512]:
(0.07+0.07+0.07+0.075+0.07)/5

0.07100000000000001

---

## lda

In [440]:
ad1 = ad_s.iloc[:n,]
ad2 = ad_s.iloc[(n):2*n,]
ad3 = ad_s.iloc[(2*n):3*n,]
ad4 = ad_s.iloc[(3*n):4*n,]
ad5 = ad_s.iloc[(4*n):5*n,]

In [441]:
X = ad[['TV','radio','newspaper']]
Y = ad['sales']

In [442]:
from sklearn.model_selection import train_test_split
X_train_1,X_test_1,Y_train_1,Y_test_1=train_test_split(X,Y,test_size=0.5,random_state=0)

In [443]:
kfold_ad1_test = ad1
kfold_ad1_train=pd.concat([ad2,ad3,ad4,ad5])

In [444]:
X_train_1 = kfold_ad1_train[['TV','radio','newspaper']]
Y_train_1 = kfold_ad1_train['sales']
X_test_1 = kfold_ad1_test[['TV','radio','newspaper']]
Y_test_1 = kfold_ad1_test['sales']

In [445]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model_lda = LinearDiscriminantAnalysis()
model_lda2 = model_lda.fit(X_train_1,Y_train_1)
Y_pred_lda =model_lda2.predict(X_test_1)
metrics.confusion_matrix(Y_test_1, Y_pred_lda)
#X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)


array([[ 8,  1],
       [ 2, 29]], dtype=int64)

In [446]:
cm1 = metrics.confusion_matrix(Y_test_1, Y_pred_lda) # confusion matrix
cm1

array([[ 8,  1],
       [ 2, 29]], dtype=int64)

In [447]:
(cm1[1,0]+cm1[0,1]) / cm1.sum()  # lda  ad1 오분류율 

0.075

----

In [448]:
kfold_ad2_test = ad2
kfold_ad2_train=pd.concat([ad1,ad3,ad4,ad5])

In [449]:
X_train_2 = kfold_ad2_train[['TV','radio','newspaper']]
Y_train_2 = kfold_ad2_train['sales']
X_test_2 = kfold_ad2_test[['TV','radio','newspaper']]
Y_test_2 = kfold_ad2_test['sales']

In [450]:
from sklearn.model_selection import train_test_split
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(X,Y,test_size=0.5,random_state=0)

In [451]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model_lda = LinearDiscriminantAnalysis()
model_lda2 = model_lda.fit(X_train_2,Y_train_2)
Y_pred_lda_2 =model_lda2.predict(X_test_2)
metrics.confusion_matrix(Y_test_2, Y_pred_lda_2)
#X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [452]:
cm2 = metrics.confusion_matrix(Y_test_2, Y_pred_lda_2) # confusion matrix
cm2

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [453]:
(cm2[1,0]+cm1[0,1]) / cm2.sum()  # lda  ad2 오분류율 

0.05

----

In [454]:
kfold_ad3_test = ad3
kfold_ad3_train=pd.concat([ad1,ad2,ad4,ad5])

In [455]:
X = ad[['TV','radio','newspaper']]
Y = ad['sales']

In [456]:
X_train_3 = kfold_ad3_train[['TV','radio','newspaper']]
Y_train_3 = kfold_ad3_train['sales']
X_test_3 = kfold_ad3_test[['TV','radio','newspaper']]
Y_test_3 = kfold_ad3_test['sales']

In [457]:
from sklearn.model_selection import train_test_split
X_train_3,X_test_3,Y_train_3,Y_test_3=train_test_split(X,Y,test_size=0.5,random_state=0)

In [458]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model_lda = LinearDiscriminantAnalysis()
model_lda2 = model_lda.fit(X_train_3,Y_train_3)
Y_pred_lda_3 =model_lda2.predict(X_test_3)
metrics.confusion_matrix(Y_test_3, Y_pred_lda_3)
#X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [459]:
cm3 = metrics.confusion_matrix(Y_test_3, Y_pred_lda_3) # confusion matrix
cm3

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [460]:
(cm3[1,0]+cm3[0,1]) / cm3.sum()  # lda  ad3 오분류율 

0.09

----

In [461]:
kfold_ad4_test = ad4
kfold_ad4_train=pd.concat([ad1,ad2,ad3,ad5])

In [462]:
X_train_4 = kfold_ad4_train[['TV','radio','newspaper']]
Y_train_4 = kfold_ad4_train['sales']
X_test_4 = kfold_ad4_test[['TV','radio','newspaper']]
Y_test_4 = kfold_ad4_test['sales']
X_train_4,X_test_4,Y_train_4,Y_test_4=train_test_split(X,Y,test_size=0.5,random_state=0)

In [463]:
model_lda = LinearDiscriminantAnalysis()
model_lda2 = model_lda.fit(X_train_4,Y_train_4)
Y_pred_lda_4=model_lda2.predict(X_test_4)
metrics.confusion_matrix(Y_test_4, Y_pred_lda_4)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [464]:
cm4 = metrics.confusion_matrix(Y_test_4, Y_pred_lda_4) # confusion matrix
(cm4[1,0]+cm4[0,1]) / cm4.sum()

0.09

----

In [465]:
kfold_ad5_test = ad5
kfold_ad5_train=pd.concat([ad1,ad2,ad3,ad4])
X_train_5 = kfold_ad5_train[['TV','radio','newspaper']]
Y_train_5 = kfold_ad5_train['sales']
X_test_5 = kfold_ad5_test[['TV','radio','newspaper']]
Y_test_5 = kfold_ad5_test['sales']
X_train_5,X_test_5,Y_train_5,Y_test_5=train_test_split(X,Y,test_size=0.5,random_state=0)

In [466]:
model_lda = LinearDiscriminantAnalysis()
model_lda2 = model_lda.fit(X_train_5,Y_train_5)
Y_pred_lda_5=model_lda2.predict(X_test_5)
metrics.confusion_matrix(Y_test_5, Y_pred_lda_5)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [467]:
cm5 = metrics.confusion_matrix(Y_test_5, Y_pred_lda_5) # confusion matrix
(cm5[1,0]+cm5[0,1]) / cm5.sum()

0.09

#### lda 의 5-fold 평균은 0.79이다 

In [470]:
(0.075+0.09+0.09+0.09+0.05)/5

0.07899999999999999

-----

### qda

In [394]:
kfold_ad1_test = ad1
kfold_ad1_train=pd.concat([ad5,ad2,ad3,ad4])
X_train_1 = kfold_ad1_train[['TV','radio','newspaper']]
Y_train_1 = kfold_ad1_train['sales']
X_test_1 = kfold_ad1_test[['TV','radio','newspaper']]
Y_test_1 = kfold_ad1_test['sales']
X_train_1,X_test_1,Y_train_1,Y_test_1=train_test_split(X,Y,test_size=0.5,random_state=0)

In [400]:
model_qda = QuadraticDiscriminantAnalysis()
model_qda2 = model_qda.fit(X_train_1,Y_train_1)
Y_pred_qda =model_qda2.predict(X_test_1)
metrics.confusion_matrix(Y_test_1, Y_pred_qda)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [401]:
cm1 = metrics.confusion_matrix(Y_test_1, Y_pred_qda) # confusion matrix
(cm1[1,0]+cm1[0,1]) / cm1.sum()

0.09

----

In [397]:
kfold_ad2_test = ad2
kfold_ad2_train=pd.concat([ad1,ad5,ad3,ad4])
X_train_2 = kfold_ad2_train[['TV','radio','newspaper']]
Y_train_2 = kfold_ad2_train['sales']
X_test_2 = kfold_ad2_test[['TV','radio','newspaper']]
Y_test_2 = kfold_ad2_test['sales']
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(X,Y,test_size=0.5,random_state=0)

In [402]:
model_qda = QuadraticDiscriminantAnalysis()
model_qda2 = model_qda.fit(X_train_2,Y_train_2)
Y_pred_qda2 =model_qda2.predict(X_test_2)
metrics.confusion_matrix(Y_test_2, Y_pred_qda2)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [403]:
cm2 = metrics.confusion_matrix(Y_test_2, Y_pred_qda2) # confusion matrix
(cm2[1,0]+cm2[0,1]) / cm2.sum()

0.09

------

In [404]:
kfold_ad3_test = ad3
kfold_ad3_train=pd.concat([ad1,ad5,ad2,ad4])
X_train_3 = kfold_ad3_train[['TV','radio','newspaper']]
Y_train_3 = kfold_ad3_train['sales']
X_test_3 = kfold_ad3_test[['TV','radio','newspaper']]
Y_test_3 = kfold_ad3_test['sales']
X_train_3,X_test_3,Y_train_3,Y_test_3=train_test_split(X,Y,test_size=0.5,random_state=0)

In [405]:
model_qda = QuadraticDiscriminantAnalysis()
model_qda2 = model_qda.fit(X_train_3,Y_train_3)
Y_pred_qda3 =model_qda2.predict(X_test_3)
metrics.confusion_matrix(Y_test_3, Y_pred_qda3)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [406]:
cm3 = metrics.confusion_matrix(Y_test_3, Y_pred_qda3) # confusion matrix
(cm3[1,0]+cm3[0,1]) / cm3.sum()

0.09

-----

In [407]:
kfold_ad4_test = ad4
kfold_ad4_train=pd.concat([ad1,ad5,ad2,ad3])
X_train_4= kfold_ad4_train[['TV','radio','newspaper']]
Y_train_4 = kfold_ad4_train['sales']
X_test_4 = kfold_ad4_test[['TV','radio','newspaper']]
Y_test_4 = kfold_ad4_test['sales']
X_train_4,X_test_4,Y_train_4,Y_test_4=train_test_split(X,Y,test_size=0.5,random_state=0)

In [409]:
model_qda = QuadraticDiscriminantAnalysis()
model_qda2 = model_qda.fit(X_train_4,Y_train_4)
Y_pred_qda4 =model_qda2.predict(X_test_4)
metrics.confusion_matrix(Y_test_4, Y_pred_qda4)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [410]:
cm4 = metrics.confusion_matrix(Y_test_4, Y_pred_qda4) # confusion matrix
(cm4[1,0]+cm4[0,1]) / cm4.sum()

0.09

---

In [411]:
kfold_ad5_test = ad5
kfold_ad5_train=pd.concat([ad1,ad4,ad2,ad3])
X_train_5= kfold_ad5_train[['TV','radio','newspaper']]
Y_train_5 = kfold_ad5_train['sales']
X_test_5 = kfold_ad5_test[['TV','radio','newspaper']]
Y_test_5 = kfold_ad5_test['sales']
X_train_5,X_test_5,Y_train_5,Y_test_5=train_test_split(X,Y,test_size=0.5,random_state=0)

In [412]:
model_qda = QuadraticDiscriminantAnalysis()
model_qda2 = model_qda.fit(X_train_5,Y_train_5)
Y_pred_qda5 =model_qda2.predict(X_test_5)
metrics.confusion_matrix(Y_test_5, Y_pred_qda5)

array([[20,  5],
       [ 4, 71]], dtype=int64)

In [413]:
cm5 = metrics.confusion_matrix(Y_test_5, Y_pred_qda5) # confusion matrix
(cm5[1,0]+cm5[0,1]) / cm5.sum()

0.09

#### qda 5-fold의 평균은 0.09이다.

----

## knn

In [421]:
kfold_ad1_test = ad1
kfold_ad1_train=pd.concat([ad5,ad2,ad3,ad4])
X_train_1 = kfold_ad1_train[['TV','radio','newspaper']]
Y_train_1 = kfold_ad1_train['sales']
X_test_1 = kfold_ad1_test[['TV','radio','newspaper']]
Y_test_1 = kfold_ad1_test['sales']
X_train_1,X_test_1,Y_train_1,Y_test_1=train_test_split(X,Y,test_size=0.5,random_state=0)

In [422]:
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_1, Y_test_1)
Y_pred_knn =model_knn.predict(X_test_1)
metrics.confusion_matrix(Y_test_1, Y_pred_knn)

array([[ 0, 25],
       [18, 57]], dtype=int64)

In [425]:
cm1 = metrics.confusion_matrix(Y_test_1, Y_pred_knn) # confusion matrix
(cm1[1,0]+cm1[0,1]) / cm1.sum()

0.43

----

In [426]:
kfold_ad2_test = ad2
kfold_ad2_train=pd.concat([ad5,ad1,ad3,ad4])
X_train_2 = kfold_ad2_train[['TV','radio','newspaper']]
Y_train_2 = kfold_ad2_train['sales']
X_test_2 = kfold_ad2_test[['TV','radio','newspaper']]
Y_test_2 = kfold_ad2_test['sales']
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(X,Y,test_size=0.5,random_state=0)

In [428]:
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_2, Y_test_2)
Y_pred_knn2 =model_knn.predict(X_test_2)
metrics.confusion_matrix(Y_test_2, Y_pred_knn2)

array([[ 0, 25],
       [18, 57]], dtype=int64)

In [429]:
cm2 = metrics.confusion_matrix(Y_test_2, Y_pred_knn2) # confusion matrix
(cm2[1,0]+cm2[0,1]) / cm2.sum()

0.43

-----

In [430]:
kfold_ad3_test = ad3
kfold_ad3_train=pd.concat([ad5,ad1,ad2,ad4])
X_train_3 = kfold_ad3_train[['TV','radio','newspaper']]
Y_train_3 = kfold_ad3_train['sales']
X_test_3 = kfold_ad3_test[['TV','radio','newspaper']]
Y_test_3 = kfold_ad3_test['sales']
X_train_3,X_test_3,Y_train_3,Y_test_3=train_test_split(X,Y,test_size=0.5,random_state=0)

In [431]:
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_3, Y_test_3)
Y_pred_knn3 =model_knn.predict(X_test_3)
metrics.confusion_matrix(Y_test_3, Y_pred_knn3)

array([[ 0, 25],
       [18, 57]], dtype=int64)

In [432]:
cm3 = metrics.confusion_matrix(Y_test_3, Y_pred_knn3) # confusion matrix
(cm3[1,0]+cm3[0,1]) / cm3.sum()

0.43

----

In [434]:
kfold_ad4_test = ad4
kfold_ad4_train=pd.concat([ad5,ad1,ad2,ad3])
X_train_4 = kfold_ad4_train[['TV','radio','newspaper']]
Y_train_4 = kfold_ad4_train['sales']
X_test_4 = kfold_ad4_test[['TV','radio','newspaper']]
Y_test_4 = kfold_ad4_test['sales']
X_train_4,X_test_4,Y_train_4,Y_test_4=train_test_split(X,Y,test_size=0.5,random_state=0)

In [435]:
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_4, Y_test_4)
Y_pred_knn4 =model_knn.predict(X_test_4)
metrics.confusion_matrix(Y_test_4, Y_pred_knn4)

array([[ 0, 25],
       [18, 57]], dtype=int64)

In [436]:
cm4 = metrics.confusion_matrix(Y_test_4, Y_pred_knn4) # confusion matrix
(cm4[1,0]+cm4[0,1]) / cm4.sum()

0.43

----

In [437]:
kfold_ad5_test = ad5
kfold_ad5_train=pd.concat([ad4,ad1,ad2,ad3])
X_train_5 = kfold_ad5_train[['TV','radio','newspaper']]
Y_train_5 = kfold_ad5_train['sales']
X_test_5 = kfold_ad5_test[['TV','radio','newspaper']]
Y_test_5 = kfold_ad5_test['sales']
X_train_5,X_test_5,Y_train_5,Y_test_5=train_test_split(X,Y,test_size=0.5,random_state=0)

In [438]:
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_5, Y_test_5)
Y_pred_knn5 =model_knn.predict(X_test_5)
metrics.confusion_matrix(Y_test_5, Y_pred_knn5)

array([[ 0, 25],
       [18, 57]], dtype=int64)

In [439]:
cm5= metrics.confusion_matrix(Y_test_5, Y_pred_knn5) # confusion matrix
(cm5[1,0]+cm5[0,1]) / cm5.sum()

0.43

#### qda 5-fold의 평균은 0.43이다

----

In [513]:
model1 = LogisticRegression()   # model instance setup
model2 = model1.fit(X,Y)      # model fitting to data
print(model2.intercept_)
print(model2.coef_)

[-21.16756665]
[[ 0.2054616   0.39803989 -0.00823107]]


In [514]:
from sklearn import metrics
Y_pred = model2.predict(X)

In [515]:
cm = metrics.confusion_matrix(Y, Y_pred) # confusion matrix
cm

array([[ 42,   3],
       [  3, 152]], dtype=int64)

In [517]:
pd.crosstab(Y,Y_pred)

col_0,0,1
sales,,
0,42,3
1,3,152


In [518]:
sum((Y_pred == 1) & (Y==1)) / sum(Y == 1) # sensitivity

0.9806451612903225

In [519]:
sum((Y_pred == 0) & (Y==0)) / sum(Y == 0) # specificity

0.9333333333333333